[박현우] 간단한 클러스터링 예제
========================

이 노트북에서는 클러스터링을 하는 방식을 간단하게 다뤄봅니다.

In [1]:
%matplotlib inline
from IPython.display import display, HTML

import matplotlib
krfont = {'family' : 'nanumgothic', 'weight' : 'bold', 'size'   : 10}
matplotlib.rc('font', **krfont)

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
from multiprocessing import Pool
import multiprocessing

cores = multiprocessing.cpu_count()

In [4]:
chosun_data = pd.read_pickle("chosun_news.df")

1. Corpus 로딩하기
---------------------

In [5]:
from konlpy.tag import Mecab, Hannanum, Twitter
mecab = Mecab()
hannanum = Hannanum()
twitter = Twitter()

In [6]:
import re, hanja
import numba

def tokenizeText(text):
    text = ' '.join(hanja.split_hanja(text))
    text = hanja.translate(text, 'substitution')
    return mecab.nouns(text)

print(tokenizeText('[사설] 美가 증거 내밀자 수사 나선 中 이중 플레이'))

['사설', '미', '증거', '수사', '나선', '중', '이중', '플레이']


In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)

In [8]:
from gensim import corpora

In [9]:
words = []
# using mecab
with Pool(cores) as p:
    words = list(tqdm(p.imap(tokenizeText, chosun_data['title']), total=len(chosun_data)))
# words = [tokenizeText(text) for text in tqdm(chosun_data['content'])]
dictionary = corpora.Dictionary(words)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


2. Doc2Vec를 이용한 문서->벡터 변환
------------------------------

In [10]:
counts = {}
for sentence in tqdm(words):
    for word in sentence:
        counts[word] = counts.get(word, 0) + 1

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [11]:
counts['의']

18

In [12]:
import gensim
from gensim.models.word2vec import Word2Vec, Text8Corpus
from collections import OrderedDict
import itertools

In [13]:
import random

sentences = [gensim.models.doc2vec.LabeledSentence(
        words=words, tags=[i])
        for i, words in enumerate(words)]
tmp = [(i, sentence) for i, sentence in enumerate(sentences)]
random.shuffle(tmp)
train_set = [x[1] for x in tmp[:len(tmp) * 3 // 10]]

In [14]:
w2v_model = Word2Vec(size=64, workers=cores)
w2v_model.build_vocab(words)

In [15]:
w2v_model.train(words)

1000127

In [16]:
w2v_model.similarity('이명박', '대통령')

0.72399169342167857

* paragraph2vec의 알고리즘에서 word2vec를 쓴다.
* Word2Vec은 전체 뉴스에 대해서 하는게 어떤가. 너무 word/content 갯수가 적지 않은가.
* 전처리 과정이 필요한가?

3. sklearn을 이용한 Agglomerative Clustering
------------------------------------------

In [17]:
def vectorizeWords(ws):
    vsum = np.zeros(64)
    fsum = 0.0
    for w in ws:
        if w in w2v_model.vocab:
            f = counts[w]
            if f == 0:
                print(w)
            fsum += 1 / f
            vsum += w2v_model[w] / f
    if fsum < 0.001:
        return np.zeros(64)
    else:
        return vsum / fsum

In [18]:
vectorizeWords(['사설', '미', '증거', '수사', '나선', '중', '이중', '플레이'])

array([ 0.02326744, -0.07676168,  0.03696233,  0.00987689,  0.05502864,
        0.05866513, -0.01148869,  0.04847459, -0.16405456,  0.0139331 ,
        0.16142796, -0.00276149, -0.02363969,  0.11010678, -0.05427916,
        0.06599977,  0.15847055, -0.1199093 , -0.02340623, -0.05826076,
        0.01110925,  0.04290614, -0.11823361,  0.12032168,  0.05085335,
        0.04675043, -0.0072211 ,  0.15575176,  0.12219956,  0.08567622,
        0.02464284, -0.09451726,  0.01727629, -0.03601583,  0.0477478 ,
       -0.04698411,  0.19029253,  0.11677974,  0.11509391, -0.09544056,
        0.05337664,  0.0851169 , -0.1139282 , -0.11747369, -0.02468912,
       -0.02794749, -0.090407  , -0.11354564,  0.06161017, -0.04227237,
       -0.07465349,  0.00030149,  0.06177193,  0.13517344,  0.11186998,
       -0.03922398, -0.02496146,  0.11335936, -0.06517375,  0.07318591,
        0.03461071, -0.01050999,  0.08935292,  0.04567676])

In [19]:
content_vectors = []
for i, sentence in enumerate(tqdm(sentences)):
    v = vectorizeWords(sentence.words)
    content_vectors.append(v)
content_vectors = np.asarray(content_vectors)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [20]:
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, SpectralClustering, MeanShift
from spherecluster import SphericalKMeans

In [21]:
cluster_num = 50
# cluster_model = AgglomerativeClustering(n_clusters=cluster_num, linkage="average", affinity='cosine')
# cluster_model = DBSCAN(eps=.30, min_samples=3, metric='cosine', algorithm='brute')
# cluster_model = KMeans(n_clusters=cluster_num)
cluster_model = SphericalKMeans(n_clusters=cluster_num)
# cluster_model = SpectralClustering(n_clusters=cluster_num, affinity='cosine')
# cluster_model = MeanShift(n_jobs=cores)
cluster_model.fit(content_vectors)

cluster_num = len(set(cluster_model.labels_)) - (1 if -1 in cluster_model.labels_ else 0)
print('# of clusters : %d' % cluster_num)
if -1 in cluster_model.labels_:
    noiseNum = np.sum(cluster_model.labels_ == -1)
    totNum = len(content_vectors)
    print('Noise : %d/%d (%.2f%%)' % (noiseNum, totNum, noiseNum / totNum * 100))

# of clusters : 50


In [22]:
cluster_data = []
for cluster_index in range(cluster_num):
    cluster_data.append(chosun_data[cluster_model.labels_ == cluster_index])

In [ ]:
cluster_model.cluster_centers_[0]

array([ 0.00906142, -0.11755657,  0.03440735,  0.01343728,  0.08631227,
        0.08654681, -0.00665735,  0.10412623, -0.25305292,  0.04015826,
        0.25743564, -0.00388694, -0.04156394,  0.1668861 , -0.08814424,
        0.10953267,  0.23459355, -0.17656178, -0.03165278, -0.05350388,
        0.03358564,  0.05574028, -0.1952626 ,  0.17438948,  0.07549638,
        0.05782397, -0.01364027,  0.22175321,  0.1757884 ,  0.11781355,
        0.05328186, -0.14409947,  0.02025538, -0.0455035 ,  0.07910068,
       -0.07356098,  0.29458924,  0.16772723,  0.15780507, -0.12697357,
        0.07419625,  0.12043047, -0.17074333, -0.18910623, -0.01703443,
       -0.05430681, -0.12772772, -0.16863227,  0.09750941, -0.06981467,
       -0.1114451 , -0.01439235,  0.12030905,  0.1985726 ,  0.14701027,
       -0.0631576 , -0.04437923,  0.15126495, -0.10236627,  0.12339782,
        0.04960776, -0.01291204,  0.13977099,  0.07583495])

In [ ]:
for i in range(cluster_num):
    print('Cluster #%d' % i)
    
    display(list(filter(
                lambda x: counts[x[0]] < 100,
                w2v_model.most_similar(
                    positive=[cluster_model.cluster_centers_[i]],
                    topn=10))))
    display(cluster_data[i][['category', 'title']].sample(10))

Cluster #0


[('미군', 0.9992335438728333),
 ('지정', 0.9989767074584961),
 ('절반', 0.9989298582077026),
 ('회계', 0.998836874961853),
 ('작전', 0.9988340139389038),
 ('합병', 0.9988318085670471),
 ('층', 0.9987736344337463),
 ('러시아', 0.9987475872039795),
 ('독일', 0.9986741542816162)]

,category,title
29512,국제,[글로벌 콕콕] 시크교도 美대위 터번·수염 소송 승리 외
35652,사회,타는 자와 걷는 자 '아찔한 동행'
44779,사회 > 국방,"南 확성기 방송 이후 ""北, 삐라 100만장 살포"""
665,사회 > 교육ㆍ시험,부동산 반짝 뜨자… 공인중개사 시험에 19만명 몰려
51980,스포츠ㆍ연예 > 스포츠 > 종합 > 동계,"'아이언 맨' 윤성빈, 金같은 銅"
5103,사회,梨大사태 23일… '느린 민주주의'의 비극
60207,사회 > 인물 > 사람들,히딩크 첫 방북… 오늘 평양 풋살구장 착공
69307,사설ㆍ칼럼 > 내부칼럼 > 오늘의 100자평,"[오늘의 100자평] ""문재인 '학자금 대출 금리 0%로'… 靑年 대책 곧 발표"""
1470,사회 > 법원ㆍ검찰ㆍ경찰,"홍만표·우병우 변론 '도나도나 사건'… 大法 ""유죄 취지로 다시 재판하라"""
11201,사회 > 법원ㆍ검찰ㆍ경찰,"[핫코너] 모기는 알고있다, 누가 범인인지"


Cluster #1


[('상상', 0.9972658157348633),
 ('순간', 0.9971310496330261),
 ('리뷰', 0.9970732927322388),
 ('죽음', 0.9968907237052917),
 ('인간', 0.9966734647750854),
 ('언니', 0.996556282043457),
 ('피부', 0.995757520198822),
 ('진짜', 0.9957261085510254)]

...

[('스트레스', 0.9985671043395996),
 ('조절', 0.9985244274139404),
 ('파악', 0.998429536819458),
 ('조합', 0.9981520771980286),
 ('기능', 0.9980584383010864),
 ('인공지능', 0.9980407953262329),
 ('자극', 0.998038649559021),
 ('세포', 0.998026967048645),
 ('전수', 0.9979896545410156),
 ('소재', 0.9979875087738037)]

,category,title
65463,경제 > 유통ㆍ소비자,고객 수면 습관 측정… 최적의 수면 환경 컨설팅
53466,사회 > 교육ㆍ시험 > 맛있는 공부,2016 수능 만점자 4人 어떻게 공부했을까?
33608,사설ㆍ칼럼 > 외부칼럼 > 최재천의 자연과 문화,[최재천의 자연과 문화] [358] 무서운 건 여전히 사람이다
6133,문화 > 북스 > 신간소개,인간의 本性은 없다… 그때그때 달라질 뿐
49635,사설ㆍ칼럼 > 외부칼럼 > 김대식 브레인,[김대식의 브레인 스토리] [167] 12월의 메시지
54906,사회 > 교육ㆍ시험 > 맛있는 공부,자녀 공부 의욕 일깨워주는 SKY 재학생 멘토링 캠프 41기 外
34795,스포츠ㆍ연예 > 엔터테인먼트 > TV/방송 > 시사/교양,[TV조선 ch19] 총선 전 여의도 정가 탐색
6470,사설ㆍ칼럼 > 외부칼럼 > 일사일언,"[일사일언] 여름 보양식의 왕, 민어"
37893,경제,"스마트폰, 가상현실에서 미래를 봤다"
12576,경제 > 유통ㆍ소비자,'힐빙' 분야 융복합 창의 전문 인력 양성의 요람


Cluster #38


[('역대', 0.9967982769012451),
 ('감축', 0.9959598183631897),
 ('법인세', 0.995909571647644),
 ('수당', 0.9958556294441223),
 ('최소', 0.9956035614013672),
 ('자금', 0.9952266216278076),
 ('정기', 0.9948452711105347),
 ('사흘', 0.9948326349258423),
 ('사기', 0.9946725368499756)]

,category,title
21777,국제 > 미국ㆍ중남미,점령군처럼… 上下院 훑은 트럼프
13645,경제,朴정부 3번째 추경… 어정쩡한 '불쏘시개'로 景氣 살릴까
24857,사회,'5% 주식룰' 탓에… 200억 기부에 세금 140억
23819,경제 > 기업,"현대重, 다음주 희망퇴직… 3000명 줄이기로"
940,국제 > 미국ㆍ중남미,"美 역대 대통령 44명 중 4명 재임중 病死… 9대 대통령 해리슨, 취임 한달 만에 숨져"
28458,사회 > 노동ㆍ복지,"光州 공무원노조, 법외노조 전공노 가입 의결"
20700,스포츠ㆍ연예 > 스포츠 > 축구 > 해외축구,"골대·벤치·샤워꼭지까지… ""돈 되는 건 다 팔아요"""
123120,정치 > 북한,"제2, 제3의 훙샹… 단둥에만 그런 企業 15개쯤 있다"
27723,정치 > 북한,北 노동당 36년 만의 黨대회 대표자 선출 일정 돌입
34885,사회 > 인물 > 사람들,미스코리아 60년… 역대 최고의 미인은?


In [ ]:
end = if # Intentionally left errornous